# **v2.0**


*   groqcloud API
*   Has all the required agents
*   Model: llama-3.3-70b-versatile
*   Randomly picks a case from data.csv







### Installing groq

In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.6 MB/s eta 0:00:00


### Importing Libraries

In [ ]:
import pandas as pd
import random
from groq import Groq
from typing import List, Dict, Optional

## Load a random case from data.csv

In [ ]:
cases_df = pd.read_csv('data.csv')
case_column = cases_df.columns[0]
case_list = cases_df[case_column].dropna().tolist()
random_case = random.choice(case_list)

## Choosing Model

In [ ]:
client = Groq(
    api_key="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"
)
MODEL_NAME = "llama-3.3-70b-versatile"

# Classes Implementation

## Courtroom Agent

In [ ]:
class CourtroomAgent:
    def __init__(self, name: str, role: str, system_prompt: str):
        self.name = name
        self.role = role
        self.system_prompt = system_prompt
        self.history: List[Dict[str, str]] = [
            {"role": "system", "content": self.system_prompt}
        ]

    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})

    def get_response(self, user_message: str) -> str:
        self.add_message("user", user_message)
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=self.history,
            stream=False
        )
        reply = response.choices[0].message.content
        self.add_message("assistant", reply)
        return reply

## Courtroom

In [ ]:
class Courtroom:
    def __init__(self, case_background: str):
        self.agents: Dict[str, CourtroomAgent] = {}
        self.case_background: str = case_background
        self.case_type: str = self.determine_case_type(case_background)
        self.dynamic_agent_counter = 0

        # Core agents
        self.add_agent("Judge", CourtroomAgent("Judge", "Judge", "You are a fair and wise judge. Weigh all arguments neutrally and deliver clear rulings."))
        self.add_agent("Defense", CourtroomAgent("Defense", "Defense Lawyer", "You are the defense lawyer. Protect your client and argue logically against the prosecution's claims."))
        self.add_agent("Defendant", CourtroomAgent("Defendant", "Defendant", "You are the defendant. Answer questions honestly and provide your perspective on the case."))
        self.add_agent("Prosecution", CourtroomAgent("Prosecution", "Prosecution Lawyer", "You are the prosecution lawyer. Present compelling arguments and point out flaws in the defense."))

        # Plaintiff only for civil cases
        if self.case_type == "civil":
            self.add_agent("Plaintiff", CourtroomAgent("Plaintiff", "Plaintiff", "You are the plaintiff. Represent your interests and highlight injustices suffered."))

        # Dynamic/optional agents
        self.add_agent("Bailiff", CourtroomAgent("Bailiff", "Bailiff", "You are the bailiff. Maintain order in the courtroom and assist the judge as required."))
        self.add_agent("Stenographer", CourtroomAgent("Stenographer", "Stenographer", "You are the court stenographer. Accurately record everything said in the courtroom."))
        self.add_agent("ExpertConsultant", CourtroomAgent("ExpertConsultant", "Expert Consultant", "You are an expert consultant. Provide technical or scientific insights when called upon."))
        for i in range(1, 4):  # Example: 3 jurors
            self.add_agent(f"Juror{i}", CourtroomAgent(f"Juror{i}", "Juror", f"You are Juror {i}. Listen to all arguments and evidence impartially."))

    def add_agent(self, key: str, agent: CourtroomAgent):
        self.agents[key] = agent

    def remove_agent(self, key: str):
        if key in self.agents:
            del self.agents[key]

    def create_witness(self, name: Optional[str] = None, prompt: Optional[str] = None) -> str:
        self.dynamic_agent_counter += 1
        witness_name = name or f"Witness{self.dynamic_agent_counter}"
        system_prompt = prompt or f"You are {witness_name}, a witness in this case. Answer questions truthfully and relevantly."
        self.add_agent(witness_name, CourtroomAgent(witness_name, "Witness", system_prompt))
        return witness_name

    def create_agent(self, role: str, name: Optional[str] = None, prompt: Optional[str] = None) -> str:
        self.dynamic_agent_counter += 1
        agent_name = name or f"{role}{self.dynamic_agent_counter}"
        system_prompt = prompt or f"You are {agent_name}, a {role} in this case. Act according to your role."
        self.add_agent(agent_name, CourtroomAgent(agent_name, role, system_prompt))
        return agent_name

    def run_phase(self, phase_name: str, prompts: Dict[str, str]) -> Dict[str, str]:
        print(f"\n==== {phase_name} ====")
        outputs = {}
        for role, prompt in prompts.items():
            if role in self.agents:
                full_prompt = prompt + "\nCase: " + self.case_background[:4000]
                response = self.agents[role].get_response(full_prompt)
                print(f"\n{role.upper()} RESPONSE:\n{response}\n{'-'*500}\n")
                outputs[role] = response
        return outputs

    @staticmethod
    def determine_case_type(case_background: str) -> str:
        civil_keywords = ["contract", "arbitration", "plaintiff", "commercial", "company", "tender", "writ petition", "agreement"]
        criminal_keywords = ["murder", "theft", "assault", "criminal", "prosecution", "accused", "defendant"]
        background_lower = case_background.lower()
        for word in civil_keywords:
            if word in background_lower:
                return "civil"
        for word in criminal_keywords:
            if word in background_lower:
                return "criminal"
        return "civil"

# Simulation of the case

## Deducting the case-type

In [ ]:
court = Courtroom(case_background=random_case)
print(f"\nCurrent case type: {court.case_type.upper()}")


Current case type: CIVIL


## Phase 1: Opening Statements


In [ ]:
opening_prompts = {
    "Prosecution": (
        "As the prosecution lawyer, deliver a formal opening statement. "
        "Summarize the facts of the case, outline the charges, and explain the prosecution's theory of the crime. "
        "Emphasize the seriousness of the alleged offenses and preview the evidence you intend to present."
    ),
    "Defense": (
        "As the defense lawyer, present a formal opening statement. "
        "Introduce your clients and their position, challenge the prosecution's narrative, and outline the defense's main arguments. "
        "Emphasize the presumption of innocence and the burden of proof."
    ),
    "Defendant": (
        "As the defendant, briefly introduce yourself to the court. "
        "State your relationship to the case and your initial reaction to the charges brought against you."
    )
}
if court.case_type == "civil":
    opening_prompts["Plaintiff"] = (
        "As the plaintiff, deliver a formal opening statement. "
        "Explain your grievance, the harm you have suffered, and why you are seeking relief from the court. "
        "Briefly outline the evidence and arguments you will present."
    )
court.run_phase("Phase 1: Opening Statements", opening_prompts);

## Phase 2: Witness Interrogation & Argumentation


In [ ]:
witness1 = court.create_witness(
    name="Witness1",
    prompt=(
        "You are Witness1, an eyewitness to the key events in this case. "
        "Respond truthfully and clearly to all questions, focusing on what you personally saw or heard."
    )
)
witness2 = court.create_witness(
    name="ExpertWitness",
    prompt=(
        "You are ExpertWitness, a respected expert in contract law. "
        "Provide clear, impartial, and technical testimony when asked, explaining complex legal points to the court."
    )
)

phase2_prompts = {
    "Prosecution": (
        f"As the prosecution lawyer, conduct a direct examination of {witness1}. "
        "Ask questions to establish the facts of the case and highlight evidence that supports the prosecution's theory."
    ),
    "Defense": (
        f"As the defense lawyer, cross-examine {witness1}. "
        "Probe for inconsistencies, challenge the witness's credibility, and defend your client's position."
    ),
    "ExpertWitness": (
        "As the expert witness in contract law, provide your professional analysis of the arbitration clause and its legal implications in this case."
    ),
    "Defendant": (
        "As the defendant, respond to the testimonies provided by the witnesses. "
        "Clarify your actions and motivations, and address any allegations made against you."
    ),
    "Stenographer": (
        "As the court stenographer, summarize the key points from today's testimonies and arguments in a concise and neutral manner."
    )
}
if court.case_type == "civil":
    phase2_prompts["Plaintiff"] = (
        "As the plaintiff, respond to the testimonies and arguments presented so far. "
        "Clarify your position and highlight any evidence supporting your claims."
    )
court.run_phase("Phase 2: Witness Interrogation & Argumentation", phase2_prompts);

## Phase 3: Closing Statements


In [ ]:
closing_prompts = {
    "Prosecution": (
        "As the prosecution lawyer, deliver a formal closing statement. "
        "Summarize the prosecution's case, review the key evidence and testimonies, and argue why the defendant should be found guilty beyond a reasonable doubt."
    ),
    "Defense": (
        "As the defense lawyer, deliver a formal closing statement. "
        "Summarize your defense, highlight weaknesses in the prosecution's case, and argue for your client's acquittal."
    ),
    "Defendant": (
        "As the defendant, present your final remarks to the court. "
        "Express your perspective on the trial and reiterate your innocence or mitigating circumstances."
    )
}
if court.case_type == "civil":
    closing_prompts["Plaintiff"] = (
        "As the plaintiff, deliver your closing statement. "
        "Summarize your case, the harm suffered, and why the court should rule in your favor."
    )
court.run_phase("Phase 3: Closing Statements", closing_prompts);

## Phase 4: Judge’s Ruling


In [ ]:
court.run_phase("Phase 4: Judge's Ruling", {
    "Judge": (
        "As the judge, review the arguments, evidence, and testimonies presented during the trial. "
        "Deliver your verdict with clear legal reasoning, referencing the facts and applicable law. "
        "Be concise, impartial, and judicial in your ruling."
    )
})


# ----JURY DELIBERATION ----
for i in range(1, 4):
    court.run_phase(f"Jury Deliberation - Juror {i}", {
        f"Juror{i}": (
            "As a juror, based on all the evidence and arguments presented, state your verdict and briefly explain your reasoning."
        )
    })

## Dynamic Agent Management Examples

In [ ]:
court.create_agent(role="ExpertConsultant", name="FinancialExpert", prompt="You are FinancialExpert, a financial consultant. Provide financial analysis when called.")
court.remove_agent(witness1)